# Token Broker via Azure Function (MPE Pattern)

This notebook demonstrates the **Hub & Spoke Token Broker** pattern for Fabric workspaces with **OAP (Outbound Access Policies)** enabled.

## Why This Pattern?

When OAP is enabled, Fabric cannot call `login.microsoftonline.com` directly.  
The Function App sits outside Fabric, calls Entra freely, and is reachable from Fabric via **MPE**.

```
Fabric Notebook
  │   POST /api/GetSPToken  (via MPE — no direct Entra ID call needed)
  ▼
Azure Function ──→ login.microsoftonline.com  (MSAL client_credentials → SP token)
  │
  └──→ returns { access_token, expires_in, scope, caller }
         │
Fabric uses the SP token to connect to the target resource directly
```

## Supported Target Scopes

The Function uses MSAL `acquire_token_for_client`, so **any Azure AD-protected resource** works.  
Pass the scope as `targetScope` in the POST body:

| Resource | `targetScope` value | DB token audience |
|---|---|---|
| **Azure SQL / Synapse Analytics** | `https://database.windows.net/.default` | `https://database.windows.net/` |
| **Azure SQL MI** | `https://database.windows.net/.default` | `https://database.windows.net/` |
| Azure Data Lake Storage Gen2 | `https://storage.azure.com/.default` | — |
| Azure Blob Storage | `https://storage.azure.com/.default` | — |
| Azure Management REST API | `https://management.azure.com/.default` | — |
| Power BI / Fabric REST API | `https://analysis.windows.net/powerbi/api/.default` | — |
| Microsoft Graph | `https://graph.microsoft.com/.default` | — |
| Custom App Registration | `api://<app-client-id>/.default` | — |

> **Note**: The SP stored in Key Vault must have the appropriate role/permission on each target resource.


In [ ]:
from notebookutils import mssparkutils
import requests
import json

# ═══════════════════════════════════════════════════════════════
# CONFIGURATION - Update these values for your deployment
# ═══════════════════════════════════════════════════════════════

# Function App's Entra app registration Client ID
FUNC_APP_CLIENT_ID = "your-function-app-client-id-here"

# Function App URL (reachable from Fabric via MPE)
FUNCTION_URL = "https://<your-function-app-name>.azurewebsites.net/api/GetSPToken"

# Target resource scope — what you want the SP token FOR
# The Function calls login.microsoftonline.com on your behalf (OAP bypass)
TARGET_SCOPE = "https://database.windows.net/.default"   # Azure SQL / Synapse

# ═══════════════════════════════════════════════════════════════


def get_sp_token(target_scope: str = TARGET_SCOPE) -> str:
    """
    Call the Hub Function to get a Service Principal token for the given scope.
    The Function bypasses OAP by calling login.microsoftonline.com on our behalf.

    Returns the access_token string ready for use.
    """
    # Step 1: Get Fabric identity token (to authenticate TO the Function)
    # Audience = Function App's App ID URI — identifies who we are to the Function
    audience = f"api://{FUNC_APP_CLIENT_ID}"
    fabric_token = mssparkutils.credentials.getToken(audience)

    # Step 2: Call the Function — it fetches the SP token from Entra/MSAL
    headers = {
        "Authorization": f"Bearer {fabric_token}",
        "Content-Type":  "application/json",
    }
    resp = requests.post(
        FUNCTION_URL,
        headers=headers,
        json={"targetScope": target_scope},
        timeout=30,
    )

    if resp.status_code != 200:
        raise RuntimeError(
            f"Function returned HTTP {resp.status_code}: {resp.text}"
        )

    body   = resp.json()
    caller = body.get("caller", {})

    print(f"SP token obtained:")
    print(f"  Scope:      {body.get('scope')}")
    print(f"  Expires in: {body.get('expires_in')}s")
    print(f"  Caller:     {caller.get('type')} — {caller.get('display')} (oid={caller.get('oid')})")

    return body["access_token"]


# Get a token for Azure SQL (used by all DB cells below)
sp_token = get_sp_token("https://database.windows.net/.default")


In [ ]:
import struct
import pyodbc
import pandas as pd

# ═══════════════════════════════════════════════════════════════
# AZURE SQL / SYNAPSE ANALYTICS — Connect using SP token
# targetScope used: https://database.windows.net/.default
# ═══════════════════════════════════════════════════════════════

SERVER   = "your-server-name.database.windows.net"
DATABASE = "database-name"


def query_sql(query: str, token: str = sp_token) -> pd.DataFrame:
    """
    Run a SQL query against Azure SQL using an AAD SP token.
    The token was obtained from the Hub Function — no direct Entra call from Fabric.

    The token audience must be https://database.windows.net/
    which is what targetScope=https://database.windows.net/.default returns.
    """
    # Pack token into the struct format pyodbc expects for AAD token auth
    token_bytes  = token.encode("utf-16-le")
    token_struct = struct.pack(f"<I{len(token_bytes)}s", len(token_bytes), token_bytes)
    SQL_COPT_SS_ACCESS_TOKEN = 1256

    conn_str = (
        "DRIVER={ODBC Driver 18 for SQL Server};"
        f"SERVER={SERVER};"
        f"DATABASE={DATABASE};"
        "Encrypt=yes;"
        "TrustServerCertificate=no;"
    )

    conn = pyodbc.connect(conn_str, attrs_before={SQL_COPT_SS_ACCESS_TOKEN: token_struct})
    df   = pd.read_sql(query, conn)
    conn.close()
    return df


# ── Example 1: Whoami (verify token works + confirm SP identity) ──
df_whoami = query_sql("SELECT SYSTEM_USER AS connected_as, DB_NAME() AS db, GETUTCDATE() AS utc_now")
display(df_whoami)

# ── Example 2: Query a table ──
df = query_sql("SELECT TOP 10 * FROM [SalesLT].[Customer]")
display(df)


In [ ]:
import requests as req

# ═══════════════════════════════════════════════════════════════
# AZURE MANAGEMENT REST API — List subscriptions using SP token
# targetScope used: https://management.azure.com/.default
# ═══════════════════════════════════════════════════════════════
arm_token = get_sp_token("https://management.azure.com/.default")

resp = req.get(
    "https://management.azure.com/subscriptions?api-version=2022-12-01",
    headers={"Authorization": f"Bearer {arm_token}"},
    timeout=30,
)
resp.raise_for_status()

subs = resp.json().get("value", [])
print(f"Subscriptions accessible by SP ({len(subs)} found):")
for s in subs:
    print(f"  {s['subscriptionId']}  {s['displayName']}  [{s['state']}]")


## Troubleshooting

| Issue | Cause | Fix |
|-------|-------|-----|
| `mssparkutils not found` | Not in Fabric | Must run inside a Fabric notebook session |
| `ConnectionError` to Function URL | MPE not active | Create & approve MPE from Fabric workspace → Function App |
| `401 Unauthorized` from Function | Wrong audience | `FUNC_APP_CLIENT_ID` must match Function App's Entra app registration |
| `403 Forbidden` from Function | MSI OID not whitelisted | Add workspace MSI OID to `allowed-msi-oids` secret in Key Vault |
| `500 Internal Error` | KV/MSAL misconfigured | Check Function App logs; verify all KV secrets exist |
| pyodbc login failed | SP has no DB access | Grant SP `db_datareader` / `db_datawriter` on the target database |
| pyodbc token struct error | Wrong scope for SQL | Use `https://database.windows.net/.default` (not management, not graph) |
| ARM API 403 | SP has no subscription access | Assign SP the required RBAC role on the subscription/resource group |

## Key Concepts

**Two tokens are always involved:**

| Token | Audience | Purpose |
|-------|----------|---------|
| Fabric token (`fabric_token`) | `api://<FUNC_APP_CLIENT_ID>` | Authenticates you **to the Function** — proves your identity |
| SP token (`sp_token`) | Target resource (e.g. `https://database.windows.net/`) | Used by Fabric to connect **to the actual resource** |

**OAP bypass explained:**
- ❌ Fabric cannot call `login.microsoftonline.com` (blocked by OAP)
- ✅ Fabric calls **Function App via MPE** (allowed)
- ✅ Function calls `login.microsoftonline.com` (outside Fabric, unrestricted)
- ✅ Fabric uses the returned SP token directly on the target resource
